# import data

In [20]:
from ElasticSearcher import search

query = {
    "term": {
        "analysis_tags.keyword": "not detect"
    }
}
data = search(query, 2101, 2201)
print('Total size of data is... %d' % len(data))

c:\users\shion\pycharmprojects\projectroute112021\venv\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Total size of data is... 17701


# Categorize data by its path

In [28]:
from DataGrouper import groupBySignature
from DataCollector import importNotDetectData

data = importNotDetectData(2101, 2201)
#ここでは、同じsignature内に同一のipからのアクセスがどれくらいあるか確認する。
dataS = groupBySignature(data)
# result1はsignatureごとに分類した上に同一destination_ipについて整理したもの
result1 = {}
for s, hits in dataS.items():
    #それぞれのIPについて分類する。
    resultIps = {}
    for hit in hits:
        if not hit['_source']['destination_ip'] in resultIps:
            resultIps[hit['_source']['destination_ip']] = []
        resultIps[hit['_source']['destination_ip']].append(hit)
    # Signature s についての分類が完了
    # データとして格納
    result1[s] = resultIps
# result1を並び変える。
# len(x[1])refers to the size of the variation of destination_ip
# ↑ if this value is bigger than 1, it means that it is prevalent
result1 = sorted(result1.items(), key=lambda x: len(dataS[x[0]]), reverse=True)
result1 = sorted(result1, key=lambda x: len(x[1]), reverse=True)
# Print and Export result1 as "signatures_by_num_destIP(1-10).csv"
f = open("resultData/signatures_by_num_destIP(2021.1.1-).csv", "wb")
# 書き出す項目について整理しておく
print("signature".center(50), "Spots", "Counts")
f.write('Signature,Spots,Hits\n'.encode('utf-8'))
for s, r in result1:
    print(s[:50].ljust(50), str(len(r)).rjust(5), str(len(dataS[s])).rjust(6))
    b = (s + ',' + str(len(r)) + ',' + str(len(dataS[s])) + '\n').encode('utf-8')
    f.write(b)
f.flush()
f.close()

c:\users\shion\pycharmprojects\projectroute112021\venv\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


                    signature                      Spots Counts
/goform/setUsbUnload/.js                              22   1117
/pages/createpage-entervariables.action               22    458
/backupmgt/localJob.php                               22    281
/v1.40/containers/create                              22    256
/                                                     21    203
/include/makecvs.php                                  21     95
/_config/query_servers/cmd                            21     79
/body.txt                                             21     27
/console/css/%252e%252e%252fconsole.portal            20     60
/tools.cgi                                            20     41
/checkValid                                           19    592
/console/css/%252e%252e/consolejndi.portal            19     42
/cgibin/mainfunction.cgi&action=login&keyPath=wget    19     19
/cgibin/mainfunction.cgi&action=login&keyPath=wget    19     19
/pages/doenterpagevariables.action      

FileNotFoundError: [Errno 2] No such file or directory: 'Memo/-goform-setUsbUnload-.js.md'